In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader, random_split, Subset
import torchvision
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
from torch.utils.data import Subset
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import torch.nn as nn
import numpy as np

class CIFAR10Classifier(nn.Module):
    def __init__(self):
        super(CIFAR10Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(6272, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(0.2 * num_train))
train_idx, test_idx = indices[split:], indices[:split]

train_dataset = Subset(trainset,train_idx)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32)
test_testset = torch.utils.data.ConcatDataset([Subset(trainset, test_idx), testset])
test_loader = torch.utils.data.DataLoader(test_testset, batch_size=64, shuffle=False)

def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Train Accuracy: {100 * correct / total}%')

    # Test loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total}%')

cuda
Files already downloaded and verified
Files already downloaded and verified


In [30]:
n_epochs = 30

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*inputs.size(0)
    epoch_loss=running_loss/len(train_loader.dataset)
    scheduler.step(epoch_loss)
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss}')
    if epoch % 5 == 4:
        evaluate(model)     
torch.save(model.state_dict(),'model.pth')

cuda
Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 0.6031436642050743
Epoch 2, Loss: 0.5934444001317024
Epoch 3, Loss: 0.5966733865857125
Epoch 4, Loss: 0.5896390744328499
Epoch 5, Loss: 0.595965623998642
Train Accuracy: 97.5475%
Test Accuracy: 64.575%
Epoch 6, Loss: 0.5916860419511795
Epoch 7, Loss: 0.6009456028938294
Epoch 00008: reducing learning rate of group 0 to 2.5000e-05.
Epoch 8, Loss: 0.5938847616076469
Epoch 9, Loss: 0.5898481168746948
Epoch 10, Loss: 0.5927182900071144
Train Accuracy: 97.68%
Test Accuracy: 64.75%
Epoch 11, Loss: 0.5884663327217102
Epoch 12, Loss: 0.5938656493544578
Epoch 13, Loss: 0.5887955960392952
Epoch 14, Loss: 0.579175396323204
Epoch 15, Loss: 0.5845593259334564
Train Accuracy: 97.7325%
Test Accuracy: 64.825%
Epoch 16, Loss: 0.5897984580874444
Epoch 17, Loss: 0.5943819497942925
Epoch 00018: reducing learning rate of group 0 to 1.2500e-05.
Epoch 18, Loss: 0.5830100085377693
Epoch 19, Loss: 0.5851366110801697


In [10]:
model_privacy = CIFAR10Classifier().to(device)
model_privacy.load_state_dict(torch.load('/kaggle/input/my-models/model_baseline.pth'))
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_privacy.parameters(), lr=0.0001,weight_decay=1e-2)
scheduler = lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

n_epochs = 40

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_privacy(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*inputs.size(0)
    epoch_loss=running_loss/len(train_loader.dataset)
    scheduler.step()
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss}')
    if epoch % 5 == 4:
        evaluate(model_privacy)
torch.save(model_privacy.state_dict(),'model_privacy_reg.pth')

Epoch 1, Loss: 0.6004154583573341
Epoch 2, Loss: 0.6423797505378723
Epoch 3, Loss: 0.6976489079475403
Epoch 4, Loss: 0.7635789583444595
Epoch 5, Loss: 0.8239570449352265
Train Accuracy: 87.84%
Test Accuracy: 66.885%
Epoch 6, Loss: 0.4890729764461517
Epoch 7, Loss: 0.4975093710899353
Epoch 8, Loss: 0.5104012539625168
Epoch 9, Loss: 0.5174827815175056
Epoch 10, Loss: 0.519823055934906
Train Accuracy: 85.085%
Test Accuracy: 68.925%
Epoch 11, Loss: 0.5035010778427124
Epoch 12, Loss: 0.5004420328378677
Epoch 13, Loss: 0.49729390136003493
Epoch 14, Loss: 0.49383416311740874
Epoch 15, Loss: 0.4899659893989563
Train Accuracy: 86.1675%
Test Accuracy: 69.325%
Epoch 16, Loss: 0.4778000099658966
Epoch 17, Loss: 0.4750052743673325
Epoch 18, Loss: 0.47233768293857575
Epoch 19, Loss: 0.4696965965151787
Epoch 20, Loss: 0.4670365028858185
Train Accuracy: 86.925%
Test Accuracy: 69.365%
Epoch 21, Loss: 0.4603620710134506
Epoch 22, Loss: 0.4587628855228424
Epoch 23, Loss: 0.45722616648674014
Epoch 24, Los

In [ ]:
poisoned_images = []
poisoned_labels = []

for i in range(len(train_dataset)):
    image, label = train_dataset[i]
    image_copy = image.clone()
    
    std_noise = 0.3
    noise = np.random.normal(size=(3, 32, 32)) * std_noise
    image_copy += noise
    poisoned_images.append(image_copy)
    poisoned_labels.append(label)
    
poisoned_images = torch.stack(poisoned_images).to(torch.float32)
poisoned_labels = torch.tensor(poisoned_labels)

class NoisyDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]
    
noisy_dataset = NoisyDataset(poisoned_images, poisoned_labels)
noisy_loader = torch.utils.data.DataLoader(noisy_dataset, batch_size = 64, shuffle=True)

model_privacy = CIFAR10Classifier().to(device)
model_privacy.load_state_dict(torch.load('/kaggle/input/my-models/model_baseline.pth'))
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_privacy.parameters(), lr=0.0001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

n_epochs = 40

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in noisy_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_privacy(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*inputs.size(0)
    epoch_loss=running_loss/len(noisy_loader.dataset)
    scheduler.step(epoch_loss)
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss}')
    if epoch % 5 == 4:
        evaluate(model_privacy)
torch.save(model_privacy.state_dict(),'model_privacy_noise.pth')

Epoch 1, Loss: 1.4049977184295654
Epoch 2, Loss: 1.2372498091697692
Epoch 3, Loss: 1.188036063671112
Epoch 4, Loss: 1.1483295279502868
Epoch 5, Loss: 1.1272585335731506
Train Accuracy: 87.635%
Test Accuracy: 59.49%
Epoch 6, Loss: 0.5784820919513702
Epoch 7, Loss: 0.500895554447174
Epoch 8, Loss: 0.46153420984745025
Epoch 9, Loss: 0.42940368609428403
Epoch 10, Loss: 0.40040717749595645
Train Accuracy: 86.935%
Test Accuracy: 60.265%
Epoch 11, Loss: 0.37296193571090697
Epoch 12, Loss: 0.34815986838340757
Epoch 13, Loss: 0.32405457615852357
Epoch 14, Loss: 0.30117551758289335
Epoch 15, Loss: 0.27965687630176544
Train Accuracy: 87.73%
Test Accuracy: 60.285%
Epoch 16, Loss: 0.25921883091926573
Epoch 17, Loss: 0.23980494067668914
Epoch 18, Loss: 0.22122882915139197
Epoch 19, Loss: 0.20337558255791663
Epoch 20, Loss: 0.18646880429387092
Train Accuracy: 87.8925%
Test Accuracy: 60.12%
Epoch 21, Loss: 0.1705017638117075
Epoch 22, Loss: 0.15522121312618256
Epoch 23, Loss: 0.1407543937921524
Epoch 

In [18]:
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()
        if epoch % 5 == 4:
            print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        correct = 0
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            correct += np.sum(predicted == labels.cpu().numpy())
            outputs=-np.sort(-outputs , axis=1)
            all_outputs.append(torch.Tensor(outputs[:,:4]))
            all_labels.append(labels)
    print('accuracy : ' , correct / len(data_loader.dataset))
    return torch.cat(all_outputs), torch.cat(all_labels)

shadow_model_count = 1

shadow_train_outputs = []
shadow_train_labels = []
shadow_test_outputs = []
shadow_test_labels = []

for _ in range(shadow_model_count):
    shadow_train_data, shadow_test_data = train_test_split(train_dataset, test_size=0.5)
    shadow_train_loader = DataLoader(shadow_train_data, batch_size=64, shuffle=False)
    shadow_test_loader = DataLoader(shadow_test_data, batch_size=64, shuffle=False)
    
    model = CIFAR10Classifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=20 , gamma = 0.7)
    train_model(model, shadow_train_loader, criterion, optimizer, scheduler , 20)
    
    train_outputs, train_labels = test_model(model, shadow_train_loader)
    shadow_train_outputs.append(train_outputs)
    shadow_train_labels.append(torch.ones_like(train_labels))  # Label 1 for seen data
    
    test_outputs, test_labels = test_model(model, shadow_test_loader)
    shadow_test_outputs.append(test_outputs)
    shadow_test_labels.append(torch.zeros_like(test_labels)) # Label 0 for un-seen data

all_train_outputs = torch.cat(shadow_train_outputs)
all_train_labels = torch.cat(shadow_train_labels)
all_test_outputs = torch.cat(shadow_test_outputs)
all_test_labels = torch.cat(shadow_test_labels)

attack_inputs = torch.cat((all_train_outputs, all_test_outputs))
attack_labels = torch.cat((all_train_labels, all_test_labels))

attack_features = attack_inputs.cpu().view(attack_inputs.size(0), -1).numpy()
attack_labels = attack_labels.cpu().numpy()

X_train, X_test, y_train, y_test = train_test_split(attack_features, attack_labels, test_size=0.2)

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 5, Loss: 1.3207589384085072
Epoch 10, Loss: 1.1258211568140755
Epoch 15, Loss: 0.9886573764462822
Epoch 20, Loss: 0.8786869955519898
accuracy :  0.8703
accuracy :  0.6152


In [19]:
from sklearn.ensemble import RandomForestClassifier

attack_model = RandomForestClassifier(max_depth=7)
attack_model.fit(X_train, y_train)

attack_predictions = attack_model.predict(X_train)
attack_accuracy = accuracy_score(y_train, attack_predictions)
print(f'Attack Model Train Accuracy: {attack_accuracy * 100:.2f}%')

attack_predictions = attack_model.predict(X_test)
attack_accuracy = accuracy_score(y_test, attack_predictions)
print(f'Attack Model Test Accuracy: {attack_accuracy * 100:.2f}%')

Attack Model Train Accuracy: 59.37%
Attack Model Test Accuracy: 56.49%


In [22]:
def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            outputs=-np.sort(-outputs , axis=1)
            all_outputs.append(torch.Tensor(outputs[:,:4]))
            all_labels.append(labels)
    return torch.cat(all_outputs), torch.cat(all_labels)

def evaluate_mia(model, attack_model, data_loader):
    model_outputs, _ = test_model(model, data_loader)
    model_features = model_outputs.cpu().view(model_outputs.size(0), -1).numpy()
    mia_predictions = attack_model.predict(model_features)
    labels = np.zeros(40000)
    labels[20000:] = 1
    return accuracy_score(mia_predictions, labels)

baseline_model = CIFAR10Classifier().to(device)
baseline_model.load_state_dict(torch.load('/kaggle/input/my-models/model_baseline.pth'))
print('baseline model : ')

privacy_model_noise = CIFAR10Classifier().to(device)
privacy_model_noise.load_state_dict(torch.load('/kaggle/input/my-models/model_privacy_noise.pth'))
print('privacy noise model')

evaluate_MIA_dataset = torch.utils.data.ConcatDataset([test_testset, Subset(train_dataset,np.arange(len(test_testset)))])
evaluate_MIA_loader = DataLoader(evaluate_MIA_dataset, batch_size = 64 , shuffle = False) 

baseline_model_predictions = evaluate_mia(baseline_model, attack_model, evaluate_MIA_loader)
privacy_noise_model_predictions = evaluate_mia(privacy_model_noise, attack_model, evaluate_MIA_loader)

print(f'Baseline Model MIA Predictions: {baseline_model_predictions}')
print(f'privacy noise Model MIA Predictions: {privacy_noise_model_predictions}')

baseline model : 
privacy noise model


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Baseline Model MIA Predictions: 0.58215
privacy noise Model MIA Predictions: 0.52555


In [28]:
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()
        if epoch % 5 == 4:
            print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        correct = 0
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            correct += np.sum(predicted == labels.cpu().numpy())
            outputs=-np.sort(-outputs , axis=1)
            all_outputs.append(torch.Tensor(outputs[:,:4]))
            all_labels.append(labels)
    print('accuracy : ' , correct / len(data_loader.dataset))
    return torch.cat(all_outputs), torch.cat(all_labels)

shadow_model_count = 3

shadow_train_outputs = []
shadow_train_labels = []
shadow_test_outputs = []
shadow_test_labels = []

for _ in range(shadow_model_count):
    shadow_train_data, shadow_test_data = train_test_split(train_dataset, test_size=0.5)
    shadow_train_loader = DataLoader(shadow_train_data, batch_size=64, shuffle=False)
    shadow_test_loader = DataLoader(shadow_test_data, batch_size=64, shuffle=False)
    
    model = CIFAR10Classifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=10 , gamma = 0.7)
    train_model(model, shadow_train_loader, criterion, optimizer, scheduler , 100)
    
    train_outputs, train_labels = test_model(model, shadow_train_loader)
    shadow_train_outputs.append(train_outputs)
    shadow_train_labels.append(torch.ones_like(train_labels))  # Label 1 for seen data
    
    test_outputs, test_labels = test_model(model, shadow_test_loader)
    shadow_test_outputs.append(test_outputs)
    shadow_test_labels.append(torch.zeros_like(test_labels)) # Label 0 for un-seen data

all_train_outputs = torch.cat(shadow_train_outputs)
all_train_labels = torch.cat(shadow_train_labels)
all_test_outputs = torch.cat(shadow_test_outputs)
all_test_labels = torch.cat(shadow_test_labels)

attack_inputs = torch.cat((all_train_outputs, all_test_outputs))
attack_labels = torch.cat((all_train_labels, all_test_labels))

attack_features = attack_inputs.cpu().view(attack_inputs.size(0), -1).numpy()
attack_labels = attack_labels.cpu().numpy()

X_train, X_test, y_train, y_test = train_test_split(attack_features, attack_labels, test_size=0.2)

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 5, Loss: 1.3088257981184572
Epoch 10, Loss: 1.1003151004687666
Epoch 15, Loss: 0.9573293552992824
Epoch 20, Loss: 0.8802372241934268
Epoch 25, Loss: 0.772881128917487
Epoch 30, Loss: 0.7408470376231038
Epoch 35, Loss: 0.6768344717856032
Epoch 40, Loss: 0.6614648207498435
Epoch 45, Loss: 0.6254129116527569
Epoch 50, Loss: 0.6030874855982991
Epoch 55, Loss: 0.5900167959947555
Epoch 60, Loss: 0.571661868796181
Epoch 65, Loss: 0.5435910688135952
Epoch 70, Loss: 0.548027363067237
Epoch 75, Loss: 0.5298273665265153
Epoch 80, Loss: 0.5296404283648481
Epoch 85, Loss: 0.5161730939397415
Epoch 90, Loss: 0.5144668611855553
Epoch 95, Loss: 0.5091935128163987
Epoch 100, Loss: 0.5069299711586949
accuracy :  0.9903
accuracy :  0.61655
Epoch 5, Loss: 1.2936482694202338
Epoch 10, Loss: 1.0889426456472744
Epoch 15, Loss: 0.9473179498800455
Epoch 20, Loss: 0.85269438706267
Epoch 25, Loss: 0.7640634867520378
Epoch 30, Loss: 0.7081203762525187
Epoch 35, Loss: 0.6616898259034933
Epoch 40, Loss: 0.6483

In [32]:
from sklearn.ensemble import RandomForestClassifier

attack_model = RandomForestClassifier(max_depth=7)
attack_model.fit(X_train, y_train)

attack_predictions = attack_model.predict(X_train)
attack_accuracy = accuracy_score(y_train, attack_predictions)
print(f'Attack Model Train Accuracy: {attack_accuracy * 100:.2f}%')

attack_predictions = attack_model.predict(X_test)
attack_accuracy = accuracy_score(y_test, attack_predictions)
print(f'Attack Model Test Accuracy: {attack_accuracy * 100:.2f}%')

Attack Model Train Accuracy: 67.66%
Attack Model Test Accuracy: 67.18%


In [33]:
def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            outputs=-np.sort(-outputs , axis=1)
            all_outputs.append(torch.Tensor(outputs[:,:4]))
            all_labels.append(labels)
    return torch.cat(all_outputs), torch.cat(all_labels)

def evaluate_mia(model, attack_model, data_loader):
    model_outputs, _ = test_model(model, data_loader)
    model_features = model_outputs.cpu().view(model_outputs.size(0), -1).numpy()
    mia_predictions = attack_model.predict(model_features)
    labels = np.zeros(40000)
    labels[20000:] = 1
    return accuracy_score(mia_predictions, labels)

baseline_model = CIFAR10Classifier().to(device)
baseline_model.load_state_dict(torch.load('/kaggle/input/my-models/model_baseline.pth'))
print('baseline model : ')

privacy_model_noise = CIFAR10Classifier().to(device)
privacy_model_noise.load_state_dict(torch.load('/kaggle/input/my-models/model_privacy_noise.pth'))
print('privacy noise model')

evaluate_MIA_dataset = torch.utils.data.ConcatDataset([test_testset, Subset(train_dataset,np.arange(len(test_testset)))])
evaluate_MIA_loader = DataLoader(evaluate_MIA_dataset, batch_size = 64 , shuffle = False) 

baseline_model_predictions = evaluate_mia(baseline_model, attack_model, evaluate_MIA_loader)
privacy_noise_model_predictions = evaluate_mia(privacy_model_noise, attack_model, evaluate_MIA_loader)

print(f'Baseline Model MIA Predictions: {baseline_model_predictions}')
print(f'privacy noise Model MIA Predictions: {privacy_noise_model_predictions}')

baseline model : 
privacy noise model


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Baseline Model MIA Predictions: 0.654238
privacy noise Model MIA Predictions: 0.537625


In [34]:
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()
        if epoch % 5 == 4:
            print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        correct = 0
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            correct += np.sum(predicted == labels.cpu().numpy())
            outputs=-np.sort(-outputs , axis=1)
            all_outputs.append(torch.Tensor(outputs[:,:4]))
            all_labels.append(labels)
    print('accuracy : ' , correct / len(data_loader.dataset))
    return torch.cat(all_outputs), torch.cat(all_labels)

shadow_model_count = 1

shadow_train_outputs = []
shadow_train_labels = []
shadow_test_outputs = []
shadow_test_labels = []

for _ in range(shadow_model_count):
    shadow_train_data, shadow_test_data = train_test_split(train_dataset, test_size=0.5)
    shadow_train_loader = DataLoader(shadow_train_data, batch_size=64, shuffle=False)
    shadow_test_loader = DataLoader(shadow_test_data, batch_size=64, shuffle=False)
    
    model = CIFAR10Classifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-2)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=10 , gamma = 0.5)
    train_model(model, shadow_train_loader, criterion, optimizer, scheduler , 50)
    
    train_outputs, train_labels = test_model(model, shadow_train_loader)
    shadow_train_outputs.append(train_outputs)
    shadow_train_labels.append(torch.ones_like(train_labels))  # Label 1 for seen data
    
    test_outputs, test_labels = test_model(model, shadow_test_loader)
    shadow_test_outputs.append(test_outputs)
    shadow_test_labels.append(torch.zeros_like(test_labels)) # Label 0 for un-seen data

all_train_outputs = torch.cat(shadow_train_outputs)
all_train_labels = torch.cat(shadow_train_labels)
all_test_outputs = torch.cat(shadow_test_outputs)
all_test_labels = torch.cat(shadow_test_labels)

attack_inputs = torch.cat((all_train_outputs, all_test_outputs))
attack_labels = torch.cat((all_train_labels, all_test_labels))

attack_features = attack_inputs.cpu().view(attack_inputs.size(0), -1).numpy()
attack_labels = attack_labels.cpu().numpy()

X_train, X_test, y_train, y_test = train_test_split(attack_features, attack_labels, test_size=0.2)

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 5, Loss: 1.541419640897562
Epoch 10, Loss: 1.4384916339057703
Epoch 15, Loss: 1.3240397166901123
Epoch 20, Loss: 1.2794586595254964
Epoch 25, Loss: 1.2065467299363863
Epoch 30, Loss: 1.1798430418434995
Epoch 35, Loss: 1.1134689861593154
Epoch 40, Loss: 1.1038193043809348
Epoch 45, Loss: 1.0648910702227024
Epoch 50, Loss: 1.0564359774985634
accuracy :  0.7603
accuracy :  0.63615


In [35]:
from sklearn.ensemble import RandomForestClassifier

attack_model = RandomForestClassifier(max_depth=7)
attack_model.fit(X_train, y_train)

attack_predictions = attack_model.predict(X_train)
attack_accuracy = accuracy_score(y_train, attack_predictions)
print(f'Attack Model Train Accuracy: {attack_accuracy * 100:.2f}%')

attack_predictions = attack_model.predict(X_test)
attack_accuracy = accuracy_score(y_test, attack_predictions)
print(f'Attack Model Test Accuracy: {attack_accuracy * 100:.2f}%')

Attack Model Train Accuracy: 57.92%
Attack Model Test Accuracy: 52.00%


In [36]:
def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            outputs=-np.sort(-outputs , axis=1)
            all_outputs.append(torch.Tensor(outputs[:,:4]))
            all_labels.append(labels)
    return torch.cat(all_outputs), torch.cat(all_labels)

def evaluate_mia(model, attack_model, data_loader):
    model_outputs, _ = test_model(model, data_loader)
    model_features = model_outputs.cpu().view(model_outputs.size(0), -1).numpy()
    mia_predictions = attack_model.predict(model_features)
    labels = np.zeros(40000)
    labels[20000:] = 1
    return accuracy_score(mia_predictions, labels)

privacy_model_reg = CIFAR10Classifier().to(device)
privacy_model_reg.load_state_dict(torch.load('/kaggle/input/my-models/model_privacy_reg.pth'))
print('privacy regulaztion model')

evaluate_MIA_dataset = torch.utils.data.ConcatDataset([test_testset, Subset(train_dataset,np.arange(len(test_testset)))])
evaluate_MIA_loader = DataLoader(evaluate_MIA_dataset, batch_size = 64 , shuffle = False) 

privacy_reg_model_predictions = evaluate_mia(privacy_model_reg, attack_model, evaluate_MIA_loader)

print(f'privacy regulaztion MIA Predictions: {privacy_reg_model_predictions}')

privacy regulaztion model


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


privacy regulaztion MIA Predictions: 0.53105


In [37]:
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()
        if epoch % 5 == 4:
            print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        correct = 0
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            correct += np.sum(predicted == labels.cpu().numpy())
            outputs=-np.sort(-outputs , axis=1)
            all_outputs.append(torch.Tensor(outputs[:,:4]))
            all_labels.append(labels)
    print('accuracy : ' , correct / len(data_loader.dataset))
    return torch.cat(all_outputs), torch.cat(all_labels)

shadow_model_count = 3

shadow_train_outputs = []
shadow_train_labels = []
shadow_test_outputs = []
shadow_test_labels = []

for _ in range(shadow_model_count):
    shadow_train_data, shadow_test_data = train_test_split(train_dataset, test_size=0.5)
    shadow_train_loader = DataLoader(shadow_train_data, batch_size=64, shuffle=False)
    shadow_test_loader = DataLoader(shadow_test_data, batch_size=64, shuffle=False)
    
    model = CIFAR10Classifier().to(device)
    model.load_state_dict(torch.load('/kaggle/input/my-models/model_baseline.pth'))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-2)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=10 , gamma = 0.5)
    train_model(model, shadow_train_loader, criterion, optimizer, scheduler , 100)
    
    train_outputs, train_labels = test_model(model, shadow_train_loader)
    shadow_train_outputs.append(train_outputs)
    shadow_train_labels.append(torch.ones_like(train_labels))  # Label 1 for seen data
    
    test_outputs, test_labels = test_model(model, shadow_test_loader)
    shadow_test_outputs.append(test_outputs)
    shadow_test_labels.append(torch.zeros_like(test_labels)) # Label 0 for un-seen data

all_train_outputs = torch.cat(shadow_train_outputs)
all_train_labels = torch.cat(shadow_train_labels)
all_test_outputs = torch.cat(shadow_test_outputs)
all_test_labels = torch.cat(shadow_test_labels)

attack_inputs = torch.cat((all_train_outputs, all_test_outputs))
attack_labels = torch.cat((all_train_labels, all_test_labels))

attack_features = attack_inputs.cpu().view(attack_inputs.size(0), -1).numpy()
attack_labels = attack_labels.cpu().numpy()

X_train, X_test, y_train, y_test = train_test_split(attack_features, attack_labels, test_size=0.2)

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 5, Loss: 1.2240301271597036
Epoch 10, Loss: 1.2689890808190782
Epoch 15, Loss: 1.1690075408917266
Epoch 20, Loss: 1.1580383145390227
Epoch 25, Loss: 1.0727429812708602
Epoch 30, Loss: 1.0396648833926874
Epoch 35, Loss: 0.9902286259130167
Epoch 40, Loss: 0.9648639546415676
Epoch 45, Loss: 0.9394692967113215
Epoch 50, Loss: 0.9223812809005713
Epoch 55, Loss: 0.9043535734898747
Epoch 60, Loss: 0.8915775937203783
Epoch 65, Loss: 0.8828917288551696
Epoch 70, Loss: 0.8814729686362294
Epoch 75, Loss: 0.8817661078974081
Epoch 80, Loss: 0.8741796477534138
Epoch 85, Loss: 0.8681775763773689
Epoch 90, Loss: 0.8705550463626179
Epoch 95, Loss: 0.8677837256425486
Epoch 100, Loss: 0.871726212029259
accuracy :  0.84365
accuracy :  0.6558
Epoch 5, Loss: 1.2253564398128765
Epoch 10, Loss: 1.2800788271922272
Epoch 15, Loss: 1.18503004045913
Epoch 20, Loss: 1.171807865746105
Epoch 25, Loss: 1.0850597968497597
Epoch 30, Loss: 1.069923601973171
Epoch 35, Loss: 1.0096930076900763
Epoch 40, Loss: 0.9834

In [38]:
from sklearn.ensemble import RandomForestClassifier

attack_model = RandomForestClassifier(max_depth=7)
attack_model.fit(X_train, y_train)

attack_predictions = attack_model.predict(X_train)
attack_accuracy = accuracy_score(y_train, attack_predictions)
print(f'Attack Model Train Accuracy: {attack_accuracy * 100:.2f}%')

attack_predictions = attack_model.predict(X_test)
attack_accuracy = accuracy_score(y_test, attack_predictions)
print(f'Attack Model Test Accuracy: {attack_accuracy * 100:.2f}%')

Attack Model Train Accuracy: 55.86%
Attack Model Test Accuracy: 55.83%


In [43]:
def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            outputs=-np.sort(-outputs , axis=1)
            all_outputs.append(torch.Tensor(outputs[:,:4]))
            all_labels.append(labels)
    return torch.cat(all_outputs), torch.cat(all_labels)

def evaluate_mia(model, attack_model, data_loader):
    model_outputs, _ = test_model(model, data_loader)
    model_features = model_outputs.cpu().view(model_outputs.size(0), -1).numpy()
    mia_predictions = attack_model.predict(model_features)
    labels = np.zeros(40000)
    labels[20000:] = 1
    return accuracy_score(mia_predictions, labels)

privacy_model_reg = CIFAR10Classifier().to(device)
privacy_model_reg.load_state_dict(torch.load('/kaggle/input/my-models/model_privacy_reg.pth'))
print('privacy regulaztion model')

evaluate_MIA_dataset = torch.utils.data.ConcatDataset([test_testset, Subset(train_dataset,np.arange(len(test_testset)))])
evaluate_MIA_loader = DataLoader(evaluate_MIA_dataset, batch_size = 64 , shuffle = False) 

privacy_reg_model_predictions = evaluate_mia(privacy_model_reg, attack_model, evaluate_MIA_loader)

print(f'privacy regulaztion MIA Predictions: {privacy_reg_model_predictions}')

privacy regulaztion model


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


privacy regulaztion MIA Predictions: 0.541247


In [44]:
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()
        if epoch % 5 == 4:
            print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

def test_model(model, data_loader):
    model.eval()
    all_outputs = [[] for i in range(10)]
    with torch.no_grad():
        correct = 0
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            correct += np.sum(predicted == labels.cpu().numpy())
            all_outputs[labels.item()].append(torch.tensor(outputs))
    print('accuracy : ' , correct / len(data_loader.dataset))
    return [torch.cat(all_outputs[i]) for i in range(10)]

shadow_model_count = 3

shadow_train_outputs = []
shadow_train_labels = []
shadow_test_outputs = []
shadow_test_labels = []

for _ in range(shadow_model_count):
    shadow_train_data, shadow_test_data = train_test_split(train_dataset, test_size=0.5)
    shadow_train_loader = DataLoader(shadow_train_data, batch_size=64, shuffle=True)
    
    model = CIFAR10Classifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=20 , gamma = 0.7)
    train_model(model, shadow_train_loader, criterion, optimizer, scheduler , 60)
    
    shadow_train_loader = DataLoader(shadow_train_data, batch_size=1, shuffle=False)
    shadow_test_loader = DataLoader(shadow_test_data, batch_size=1, shuffle=False)
    
    train_outputs = test_model(model, shadow_train_loader)
    shadow_train_outputs.append(train_outputs)

    test_outputs = test_model(model, shadow_test_loader)
    shadow_test_outputs.append(test_outputs)

attack_inputs = []
attack_labels = []
for i in range(10):
    all_train_outputs = torch.cat([shadow_train_outputs[j][i] for j in range(shadow_model_count)])
    all_test_outputs = torch.cat([shadow_test_outputs[j][i] for j in range(shadow_model_count)])
    attack_inputs.append(torch.cat((all_train_outputs, all_test_outputs)).cpu().numpy())
    attack_labels.append(torch.cat((torch.ones(all_train_outputs.shape[0]),torch.zeros(all_test_outputs.shape[0]))).cpu().numpy())

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 5, Loss: 1.3530470288980503
Epoch 10, Loss: 1.1261846908746056
Epoch 15, Loss: 0.9864334496446311
Epoch 20, Loss: 0.8874182017466512
Epoch 25, Loss: 0.7739789630658329
Epoch 30, Loss: 0.7326288106152044
Epoch 35, Loss: 0.7058166011263387
Epoch 40, Loss: 0.6667870883934033
Epoch 45, Loss: 0.6137640987531826
Epoch 50, Loss: 0.5985221499071335
Epoch 55, Loss: 0.5912525315825551
Epoch 60, Loss: 0.5663134106241476
accuracy :  0.98445
accuracy :  0.6117
Epoch 5, Loss: 1.3537294508550113
Epoch 10, Loss: 1.1419378454311968
Epoch 15, Loss: 1.0056898176860505
Epoch 20, Loss: 0.8823718763768863
Epoch 25, Loss: 0.7938769162653353
Epoch 30, Loss: 0.7321221598992332
Epoch 35, Loss: 0.6895211843636851
Epoch 40, Loss: 0.6678124989945287
Epoch 45, Loss: 0.6100456557525232
Epoch 50, Loss: 0.5877366406849017
Epoch 55, Loss: 0.5791970993192813
Epoch 60, Loss: 0.5480531800669223
accuracy :  0.9855
accuracy :  0.60925
Epoch 5, Loss: 1.3362122545607935
Epoch 10, Loss: 1.140154929397205
Epoch 15, Loss: 

In [45]:
from sklearn.ensemble import RandomForestClassifier

attack_models = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(attack_inputs[i], attack_labels[i], test_size=0.2)
    attack_model = RandomForestClassifier(max_depth=8)
    attack_model.fit(X_train, y_train)
    attack_models.append(attack_model)
    attack_predictions = attack_model.predict(X_train)
    attack_accuracy = accuracy_score(y_train, attack_predictions)
    print(f'Attack Model Train Accuracy for class {i+1}: {attack_accuracy * 100:.2f}%')
    attack_predictions = attack_model.predict(X_test)
    attack_accuracy = accuracy_score(y_test, attack_predictions)
    print(f'Attack Model Test Accuracy for class {i+1}: {attack_accuracy * 100:.2f}%')

Attack Model Train Accuracy for class 1: 74.91%
Attack Model Test Accuracy for class 1: 72.13%
Attack Model Train Accuracy for class 2: 70.74%
Attack Model Test Accuracy for class 2: 67.43%
Attack Model Train Accuracy for class 3: 82.67%
Attack Model Test Accuracy for class 3: 80.05%
Attack Model Train Accuracy for class 4: 84.60%
Attack Model Test Accuracy for class 4: 82.14%
Attack Model Train Accuracy for class 5: 82.65%
Attack Model Test Accuracy for class 5: 78.80%
Attack Model Train Accuracy for class 6: 80.90%
Attack Model Test Accuracy for class 6: 77.73%
Attack Model Train Accuracy for class 7: 72.65%
Attack Model Test Accuracy for class 7: 71.24%
Attack Model Train Accuracy for class 8: 73.86%
Attack Model Test Accuracy for class 8: 70.83%
Attack Model Train Accuracy for class 9: 72.25%
Attack Model Test Accuracy for class 9: 67.76%
Attack Model Train Accuracy for class 10: 74.44%
Attack Model Test Accuracy for class 10: 71.77%


In [47]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", module="torch")

def evaluate_mia(model, attack_model, data_loader):
    predictions = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predictions.append(attack_models[labels.item()].predict(outputs)[0])
    y_true = np.zeros(40000)
    y_true[20000:] = 1
    return accuracy_score(y_true, predictions)

baseline_model = CIFAR10Classifier().to(device)
baseline_model.load_state_dict(torch.load('/kaggle/input/my-models/model_baseline.pth'))
print('baseline model : ')

evaluate_MIA_dataset = torch.utils.data.ConcatDataset([test_testset, Subset(train_dataset,np.arange(len(test_testset)))])
evaluate_MIA_loader = DataLoader(evaluate_MIA_dataset, batch_size = 1 , shuffle = False) 

model_predictions = evaluate_mia(baseline_model, attack_model, evaluate_MIA_loader)

print(f'Baseline Model MIA Predictions: {model_predictions}')

baseline model : 
Baseline Model MIA Predictions: 0.60255


Epoch 1, Loss: 1.7124318467140198
Epoch 2, Loss: 1.4805127268314362
Epoch 3, Loss: 1.3878559970855713
Epoch 4, Loss: 1.3129268690109253
Train Accuracy: 66.2625%
Test Accuracy: 61.44%


In [10]:
model_privacy = CIFAR10Classifier().to(device)
model_privacy.load_state_dict(torch.load('/kaggle/working/model.pth'))
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_privacy.parameters(), lr=0.001,weight_decay=5e-2)
scheduler = lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.5)

n_epochs = 10

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_privacy(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*inputs.size(0)
    epoch_loss=running_loss/len(train_loader.dataset)
    scheduler.step()
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss}')
    if epoch % 5 == 4:
        evaluate(model_privacy)
torch.save(model_privacy.state_dict(),'model_privacy_reg.pth')

Epoch 1, Loss: 1.6710456382274628
Epoch 2, Loss: 1.7742864512443544
Epoch 3, Loss: 1.7871973566055297
Epoch 4, Loss: 1.7303930786132813
Epoch 5, Loss: 1.7140724746704101
Train Accuracy: 45.03%
Test Accuracy: 44.785%
Epoch 6, Loss: 1.5142632371902467
Epoch 7, Loss: 1.4439791026115418
Epoch 8, Loss: 1.4228863829135894
Epoch 9, Loss: 1.4093726710796357
Epoch 10, Loss: 1.37401756272316
Train Accuracy: 52.535%
Test Accuracy: 52.365%


In [ ]:
poisoned_images = []
poisoned_labels = []

for i in range(len(train_dataset)):
    image, label = train_dataset[i]
    image_copy = image.clone()
    
    std_noise = 0.3
    noise = np.random.normal(size=(3, 32, 32)) * std_noise
    image_copy += noise
    poisoned_images.append(image_copy)
    poisoned_labels.append(label)
    
poisoned_images = torch.stack(poisoned_images).to(torch.float32)
poisoned_labels = torch.tensor(poisoned_labels)

class NoisyDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]
    
noisy_dataset = NoisyDataset(poisoned_images, poisoned_labels)
noisy_loader = torch.utils.data.DataLoader(noisy_dataset, batch_size = 64, shuffle=True)

model_privacy = CIFAR10Classifier().to(device)
model_privacy.load_state_dict(torch.load('/kaggle/input/my-models/model_baseline.pth'))
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_privacy.parameters(), lr=0.0001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

n_epochs = 40

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in noisy_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_privacy(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*inputs.size(0)
    epoch_loss=running_loss/len(noisy_loader.dataset)
    scheduler.step(epoch_loss)
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss}')
    if epoch % 5 == 4:
        evaluate(model_privacy)
torch.save(model_privacy.state_dict(),'model_privacy_noise.pth')

In [29]:
pip install opacus

Note: you may need to restart the kernel to use updated packages.


In [31]:
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from torch.optim import lr_scheduler

# Define the model, loss function and optimizer
model = CIFAR10Classifier().to(device)
model.load_state_dict(torch.load('kim_model.pth'))
model = ModuleValidator.fix(model)  # Ensure model is compatible with Opacus
ModuleValidator.validate(model, strict=False)  # Validate model for DP

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

# Initialize Privacy Engine
privacy_engine = PrivacyEngine()

# Attach Privacy Engine
model, optimizer, trainloader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=trainloader,
    noise_multiplier=1.1,
    max_grad_norm=1.0,
)

# Training loop
n_epochs = 50

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(trainloader.dataset)
    scheduler.step()
    if epoch % 5 == 4:
        evaluate()
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss}')

evaluate()


Epoch 1, Loss: 1.4772104330033065
Epoch 2, Loss: 1.8108938303336501
Epoch 3, Loss: 1.8552120708599686
Epoch 4, Loss: 1.9156563150823116
Train Accuracy: 51.32494079521376%
Test Accuracy: 50.635%
Epoch 5, Loss: 1.9491818628579378
Epoch 6, Loss: 1.9334798613637685
Epoch 7, Loss: 1.9002029766261577
Epoch 8, Loss: 1.8782178087115289
Epoch 9, Loss: 1.8721493428081275
Train Accuracy: 53.18032786885246%
Test Accuracy: 51.595%
Epoch 10, Loss: 1.837915560400486
Epoch 11, Loss: 1.8417416595995426
Epoch 12, Loss: 1.8423255498617888
Epoch 13, Loss: 1.8168749167650937
Epoch 14, Loss: 1.8146780160337685
Train Accuracy: 53.12209463024851%
Test Accuracy: 51.815%
Epoch 15, Loss: 1.8465488351583481
Epoch 16, Loss: 1.810504553720355
Epoch 17, Loss: 1.8204223701387643
Epoch 18, Loss: 1.8089205376565456
Epoch 19, Loss: 1.822652270373702
Train Accuracy: 52.98313714616994%
Test Accuracy: 51.825%
Epoch 20, Loss: 1.8102970042705535
Epoch 21, Loss: 1.8034724285811186
Epoch 22, Loss: 1.81887153685987
Epoch 23, Lo

In [33]:
torch.save(model.state_dict(),'kim_model_privacy.pth')

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import datasets, transforms
import numpy as np
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from torch.optim import lr_scheduler
import torch.optim as optim
from copy import deepcopy

# Prepare the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split training data into seen (80%) and unseen (20%)
num_train = len(trainset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(0.8 * num_train))
seen_indices, unseen_indices = indices[:split], indices[split:]

seen_set = Subset(trainset, seen_indices)
unseen_set = Subset(trainset, unseen_indices)

# Loaders
seen_loader = DataLoader(seen_set, batch_size=32, shuffle=True)
unseen_loader = DataLoader(unseen_set, batch_size=32, shuffle=True)
test_loader = DataLoader(testset, batch_size=32, shuffle=False)

# Define the model, loss function and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load and evaluate the baseline model
baseline_model = CIFAR10Classifier().to(device)
baseline_model.load_state_dict(torch.load('/kaggle/working/kim_model.pth'))
baseline_model.eval()

# Load and evaluate the privacy-enhanced model
privacy_model = CIFAR10Classifier().to(device)
privacy_model.load_state_dict(torch.load('/kaggle/working/kim_model_privacy.pth', map_location=device), strict=False)
privacy_model.eval()

# Helper function to get model predictions
def get_predictions(model, loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            predictions.append(outputs.detach().cpu())
    return torch.cat(predictions)

# Get predictions from both models on seen and unseen data
baseline_seen_preds = get_predictions(baseline_model, seen_loader)
baseline_unseen_preds = get_predictions(baseline_model, unseen_loader)

privacy_seen_preds = get_predictions(privacy_model, seen_loader)
privacy_unseen_preds = get_predictions(privacy_model, unseen_loader)


# Prepare the attacker data
def prepare_attacker_data(seen_preds, unseen_preds):
    seen_labels = torch.ones(seen_preds.size(0))
    unseen_labels = torch.zeros(unseen_preds.size(0))
    return torch.cat([seen_preds, unseen_preds]), torch.cat([seen_labels, unseen_labels])

baseline_attacker_data, baseline_attacker_labels = prepare_attacker_data(baseline_seen_preds, baseline_unseen_preds)
privacy_attacker_data, privacy_attacker_labels = prepare_attacker_data(privacy_seen_preds, privacy_unseen_preds)

# Define a more complex Attacker model
class ComplexAttackerModel(nn.Module):
    def __init__(self, input_size):
        super(ComplexAttackerModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

# Train the attacker model
def train_attacker(attacker_model, data, labels):
    attacker_model = attacker_model.to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(attacker_model.parameters(), lr=0.001)

    dataset = torch.utils.data.TensorDataset(data, labels)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    attacker_model.train()
    for epoch in range(50):  # Train for 50 epochs
        running_loss = 0.0
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = attacker_model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(loader)}")

    return attacker_model

# Instantiate and train attacker models
input_size = baseline_attacker_data.size(1)
baseline_attacker = ComplexAttackerModel(input_size)
privacy_attacker = ComplexAttackerModel(input_size)

print("Training baseline attacker...")
baseline_attacker = train_attacker(baseline_attacker, baseline_attacker_data, baseline_attacker_labels)

print("Training privacy attacker...")
privacy_attacker = train_attacker(privacy_attacker, privacy_attacker_data, privacy_attacker_labels)

# Evaluate the attacker models
def evaluate_attacker(attacker_model, data, labels):
    attacker_model.eval()
    with torch.no_grad():
        outputs = attacker_model(data.to(device)).squeeze()
        preds = (outputs > 0.5).float().cpu()
        accuracy = (preds == labels).float().mean().item()
    return accuracy

baseline_attacker_accuracy = evaluate_attacker(baseline_attacker, baseline_attacker_data, baseline_attacker_labels)
privacy_attacker_accuracy = evaluate_attacker(privacy_attacker, privacy_attacker_data, privacy_attacker_labels)

print(f"Baseline Attacker Accuracy: {baseline_attacker_accuracy * 100:.2f}%")
print(f"Privacy Attacker Accuracy: {privacy_attacker_accuracy * 100:.2f}%")

Files already downloaded and verified
Files already downloaded and verified
Baseline seen predictions: tensor([[-2.4831, -1.2110,  1.7662,  1.3163,  0.6339,  0.7486,  2.0891, -0.2556,
         -2.2281, -0.9528],
        [ 0.4810, -0.6539,  0.8898, -0.6647,  2.4522, -0.8908, -0.0288, -0.1195,
         -0.8072, -1.0280],
        [ 0.6598,  2.3060, -1.1582, -0.9929, -0.5425, -1.5622, -0.0236, -1.5446,
          0.9652,  1.7259],
        [-1.1429, -1.0228,  1.2568, -0.1124,  3.1408, -0.0551,  1.3397,  0.4053,
         -2.7172, -1.7134],
        [-2.3763, -2.5745,  1.2090,  2.7858,  0.6261,  2.5957,  0.9766,  0.2477,
         -2.1618, -1.8062]])
Baseline unseen predictions: tensor([[-0.6497, -0.4208,  0.2956,  0.6390,  0.5124,  0.2382, -0.0663, -0.0159,
         -0.4348, -0.2425],
        [-1.0127, -3.7558,  3.1709,  0.7084,  2.3261,  1.9243, -0.5121,  2.0058,
         -3.0185, -2.3087],
        [ 1.0644,  1.2032, -1.0783, -1.1890, -1.6148, -1.7972, -1.7374, -1.1041,
          2.2862,  3.73